<link rel="stylesheet" href="/site-assets/css/gemma.css">
<link rel="stylesheet" href="https://fonts.googleapis.com/css2?family=Google+Symbols:opsz,wght,FILL,GRAD@20..48,100..700,0..1,-50..200" />

In [ ]:
import os
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.

os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

### Install dependencies

Install Keras, KerasNLP, and other dependencies.

In [ ]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U "keras>=3"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.1/644.1 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 77.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.


In [ ]:
import os

os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

### Import packages

Import Keras and KerasNLP.

In [ ]:
import keras
import keras_nlp

## Load Dataset

In [ ]:
import json

# List to store the data
data = []

# Path to the JSON file
file_path = "heart_disease_qa_dataset.json"

# Open and read the JSON file
with open(file_path, "r") as file:
    data = json.load(file)  # Parse the entire JSON file
    print(json.dumps(data[:5], indent=4))  # Print first 5 entries for preview



[
    {
        "input": "I am a man of 70 years old. My chest pain type is asymptomatic, my resting blood pressure is 130 mm Hg, my cholesterol level is 322 mg/dl, my fasting blood sugar is below 120 mg/dl, the results of my EKG are 2, my maximum heart rate achieved is 109 bpm, I dont have exercise-induced angina, and my ST depression is 2.4. The slope of the ST segment is 2, the number of major vessels colored by fluoroscopy is 3, and my thallium scan result is normal.",
        "output": "Based on the provided information, you are suffering from heart disease."
    },
    {
        "input": "I am a woman of 67 years old. My chest pain type is non-anginal pain, my resting blood pressure is 115 mm Hg, my cholesterol level is 564 mg/dl, my fasting blood sugar is below 120 mg/dl, the results of my EKG are 2, my maximum heart rate achieved is 160 bpm, I dont have exercise-induced angina, and my ST depression is 1.6. The slope of the ST segment is 2, the number of major vessels colored by

In [ ]:

formatted_data = []


# Use a template to format the data
template = "Instruction:\n{input}\n\nResponse:\n{output}"

# Format each record
for record in data[:270]:  # Only process the first 1000 examples
    formatted_example = template.format(
        input=record.get("input", ""),  # Get the "input" field
        output=record.get("output", "")  # Get the "output" field
    )
    formatted_data.append(formatted_example)

# Print the total number of records processed and a preview of the formatted data
print(f"\nTotal records formatted: {len(formatted_data)}")
print("\nPreview of the first 3 formatted records:")
print("\n".join(formatted_data[:3]))



Total records formatted: 270

Preview of the first 3 formatted records:
Instruction:
I am a man of 70 years old. My chest pain type is asymptomatic, my resting blood pressure is 130 mm Hg, my cholesterol level is 322 mg/dl, my fasting blood sugar is below 120 mg/dl, the results of my EKG are 2, my maximum heart rate achieved is 109 bpm, I dont have exercise-induced angina, and my ST depression is 2.4. The slope of the ST segment is 2, the number of major vessels colored by fluoroscopy is 3, and my thallium scan result is normal.

Response:
Based on the provided information, you are suffering from heart disease.
Instruction:
I am a woman of 67 years old. My chest pain type is non-anginal pain, my resting blood pressure is 115 mm Hg, my cholesterol level is 564 mg/dl, my fasting blood sugar is below 120 mg/dl, the results of my EKG are 2, my maximum heart rate achieved is 160 bpm, I dont have exercise-induced angina, and my ST depression is 1.6. The slope of the ST segment is 2, the num

In [ ]:
formatted_data[1]

'Instruction:\nI am a woman of 67 years old. My chest pain type is non-anginal pain, my resting blood pressure is 115 mm Hg, my cholesterol level is 564 mg/dl, my fasting blood sugar is below 120 mg/dl, the results of my EKG are 2, my maximum heart rate achieved is 160 bpm, I dont have exercise-induced angina, and my ST depression is 1.6. The slope of the ST segment is 2, the number of major vessels colored by fluoroscopy is 0, and my thallium scan result is reversible defect.\n\nResponse:\nBased on the provided information, you do not have heart disease.'

In [ ]:
data[1]

{'input': 'I am a woman of 67 years old. My chest pain type is non-anginal pain, my resting blood pressure is 115 mm Hg, my cholesterol level is 564 mg/dl, my fasting blood sugar is below 120 mg/dl, the results of my EKG are 2, my maximum heart rate achieved is 160 bpm, I dont have exercise-induced angina, and my ST depression is 1.6. The slope of the ST segment is 2, the number of major vessels colored by fluoroscopy is 0, and my thallium scan result is reversible defect.',
 'output': 'Based on the provided information, you do not have heart disease.'}

In [ ]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_2b_en")
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,614,341,888 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,614,341,888 (9.74 GB)

 Trainable params: 2,614,341,888 (9.74 GB)

 Non-trainable params: 0 (0.00 B)

## Inference before fine tuning

In this section, you will query the model with various prompts to see how it responds.

In [ ]:
prompt = template.format(
    input="I am a woman of 67 years old. My chest pain type is non-anginal pain, my resting blood pressure is 115 mm Hg, my cholesterol level is 564 mg/dl, my fasting blood sugar is below 120 mg/dl, the results of my EKG are 2, my maximum heart rate achieved is 160 bpm, I dont have exercise-induced angina, and my ST depression is 1.6. The slope of the ST segment is 2, the number of major vessels colored by fluoroscopy is 0, and my thallium scan result is reversible defect.",
    output="",
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
I am a woman of 67 years old. My chest pain type is non-anginal pain, my resting blood pressure is 115 mm Hg, my cholesterol level is 564 mg/dl, my fasting blood sugar is below 120 mg/dl, the results of my EKG are 2, my maximum heart rate achieved is 160 bpm, I dont have exercise-induced angina, and my ST depression is 1.6. The slope of the ST segment is 2, the number of major vessels colored by fluoroscopy is 0, and my thallium scan result is reversible defect.

Response:
This case is very typical. The patient is a woman with a long history of chest pain. Her resting blood pressure is 115 mm Hg and the maximum heart rate is 160 bpm. The EKG is normal, but the blood cholesterol is very high. The patient also has a history of diabetes.

The patient has a history of non-anginal chest pain and the ST depression is only 1.6 mm. She does have an elevated blood cholesterol and the thallium scan is reversible, so the diagnosis is probably hypercholesterolemia. The


In [ ]:
# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,617,270,528 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,617,270,528 (9.75 GB)

 Trainable params: 2,928,640 (11.17 MB)

 Non-trainable params: 2,614,341,888 (9.74 GB)

In [ ]:
gemma_lm.preprocessor.sequence_length = 256
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)

optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(formatted_data, epochs=1, batch_size=1)

270/270 ━━━━━━━━━━━━━━━━━━━━ 271s 868ms/step - loss: 0.9120 - sparse_categorical_accuracy: 0.6195


In [ ]:
prompt = template.format(
    input="I am a woman of 67 years old. My chest pain type is non-anginal pain, my resting blood pressure is 115 mm Hg, my cholesterol level is 564 mg/dl, my fasting blood sugar is below 120 mg/dl, the results of my EKG are 2, my maximum heart rate achieved is 160 bpm, I dont have exercise-induced angina, and my ST depression is 1.6. The slope of the ST segment is 2, the number of major vessels colored by fluoroscopy is 0, and my thallium scan result is reversible defect.",
    output="",
)

sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
I am a woman of 67 years old. My chest pain type is non-anginal pain, my resting blood pressure is 115 mm Hg, my cholesterol level is 564 mg/dl, my fasting blood sugar is below 120 mg/dl, the results of my EKG are 2, my maximum heart rate achieved is 160 bpm, I dont have exercise-induced angina, and my ST depression is 1.6. The slope of the ST segment is 2, the number of major vessels colored by fluoroscopy is 0, and my thallium scan result is reversible defect.

Response:
Based on the provided information, you do not have heart disease.


In [ ]:
save_directory = "./fine_tuned_gemma_lm"

# Save model weights (ensure filename ends with `.weights.h5`)
#gemma_lm.save_weights(f"{save_directory}/gemma_lm.weights.h5")

gemma_lm.save_to_preset(save_directory)

In [ ]:
import kagglehub
import keras_hub

kaggle_username = kagglehub.whoami()["username"]
kaggle_username

Kaggle credentials successfully validated.


'marambak'

In [ ]:
kaggle_uri = "kaggle://marambak/gemma/keras/fine_tuned_gemma_lm"
keras_nlp.upload_preset(kaggle_uri, "./fine_tuned_gemma_lm")

Uploading Model https://www.kaggle.com/models/marambak/gemma/keras/fine_tuned_gemma_lm ...
Model 'gemma' does not exist or access is forbidden for user 'marambak'. Creating or handling Model...
Model 'gemma' Created.
Starting upload for file ./fine_tuned_gemma_lm/model.weights.h5


Uploading: 100%|██████████| 10.5G/10.5G [02:33<00:00, 68.0MB/s]

Upload successful: ./fine_tuned_gemma_lm/model.weights.h5 (10GB)
Starting upload for file ./fine_tuned_gemma_lm/task.json



Uploading: 100%|██████████| 2.98k/2.98k [00:00<00:00, 5.09kB/s]

Upload successful: ./fine_tuned_gemma_lm/task.json (3KB)
Starting upload for file ./fine_tuned_gemma_lm/preprocessor.json


...


Uploading: 100%|██████████| 143/143 [00:00<00:00, 223B/s]

Upload successful: ./fine_tuned_gemma_lm/metadata.json (143B)
Starting upload for file ./fine_tuned_gemma_lm/assets/tokenizer/vocabulary.spm



Uploading: 100%|██████████| 4.24M/4.24M [00:00<00:00, 6.22MB/s]

Upload successful: ./fine_tuned_gemma_lm/assets/tokenizer/vocabulary.spm (4MB)


Your model instance has been created.
Files are being processed...
See at: https://www.kaggle.com/models/marambak/gemma/keras/fine_tuned_gemma_lm


In [ ]:
causal_lm = keras_nlp.models.CausalLM.from_preset(
    "kaggle://marambak/gemma/keras/fine_tuned_gemma_lm"
)

100%|██████████| 782/782 [00:00<00:00, 447kB/s]


KeyboardInterrupt: 

In [ ]:
template = "Instruction:\n{input}\n\nResponse:\n{output}"

prompt = template.format(
    input="I am a woman of 67 years old. My chest pain type is non-anginal pain, my resting blood pressure is 115 mm Hg, my cholesterol level is 564 mg/dl, my fasting blood sugar is below 120 mg/dl, the results of my EKG are 2, my maximum heart rate achieved is 160 bpm, I dont have exercise-induced angina, and my ST depression is 1.6. The slope of the ST segment is 2, the number of major vessels colored by fluoroscopy is 0, and my thallium scan result is reversible defect.",
    output="",
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
causal_lm.compile(sampler=sampler)
print(causal_lm.generate(prompt, max_length=256))

Instruction:
I am a woman of 67 years old. My chest pain type is non-anginal pain, my resting blood pressure is 115 mm Hg, my cholesterol level is 564 mg/dl, my fasting blood sugar is below 120 mg/dl, the results of my EKG are 2, my maximum heart rate achieved is 160 bpm, I dont have exercise-induced angina, and my ST depression is 1.6. The slope of the ST segment is 2, the number of major vessels colored by fluoroscopy is 0, and my thallium scan result is reversible defect.

Response:
Based on the provided information, you do not have heart disease.


In [ ]:
import gradio as gr

# Define the function to process the input
def diagnose_symptoms(symptoms):
    # Example placeholder logic for diagnosis
    diagnosis = "Based on your symptoms, you may have 'Hypertension' or 'Non-anginal chest pain'."
    return diagnosis

# Define the Gradio interface
with gr.Blocks(theme=gr.themes.Soft()) as medical_chat:
    # Animated Title at the top
    with gr.Row():
        with gr.Column():
            gr.Image(
                value="/content/logo.PNG",
                elem_id="logo-image",
                label="",
            )

    # Chat container for input/output in the same area
    with gr.Group():  # Changed from gr.Box() to gr.Group()
        with gr.Column():
            # Input Field with placeholder
            input_box = gr.Textbox(
                label="",
                placeholder="Describe your symptoms, and we will provide you with the necessary help...",
                lines=3,
                elem_id="symptom-input",
                interactive=True,
            )

            # Submit Button in Green
            diagnose_button = gr.Button(
                "Diagnose", elem_id="diagnose-button", variant="primary"
            )

            # Output Field for Diagnosis
            output_box = gr.Textbox(
                label="",
                placeholder="Here is the diagnosis...",
                lines=5,
                elem_id="diagnosis-output",
                interactive=False,
            )

    # Set up interactions
    diagnose_button.click(
        fn=diagnose_symptoms, inputs=[input_box], outputs=[output_box]
    )

    # Add CSS for resizing the logo
css = """
#logo-image img {
    width: 40px; /* Adjust the width as needed */
    height: 40px; /* Ensure the height matches */
    object-fit: contain; /* Ensure the image is contained within the set width/height */
    margin-top: 10px;
}
"""

# Launch the app
medical_chat.launch()